<a href="https://colab.research.google.com/github/jpmachinelearning/U4_S16_Trabajo_Final_2P_PLAZA_A-PADILLA_J/blob/main/src/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyGithub

In [ ]:
from google.colab import files
from github import Github, Auth, GithubException
import pandas as pd

# ==========================================
# 1. CONFIGURACIÓN
# ==========================================
TOKEN = ''
REPO_NAME = 'scikit-learn/scikit-learn'
LIMITE_REGISTROS = 50

# ==========================================
# 2. CONEXIÓN (Modo 2026 corregido)
# ==========================================
try:
    # Nueva forma de autenticación para evitar el DeprecationWarning
    auth = Auth.Token(TOKEN)
    g = Github(auth=auth)

    repo = g.get_repo(REPO_NAME)
    print(f"✅ Conectado exitosamente a: {repo.full_name}")

    data = []

    # ==========================================
    # 3. EXTRACCIÓN DE ISSUES (RF-01)
    # ==========================================
    print("\n--- Extrayendo Issues (excluyendo PRs) ---")
    issues = repo.get_issues(state='all') # Traemos todos
    count = 0

    for item in issues:
        if count >= LIMITE_REGISTROS: break

        # FILTRO IMPORTANTE: En la API de GitHub, los PRs técnicamente son Issues.
        # Si 'pull_request' no es None, significa que es un PR y lo saltamos.
        if item.pull_request is not None:
            continue

        print(f"📥 Procesando Issue #{item.number}...")
        comments = [c.body for c in item.get_comments()]

        data.append({
            'tipo': 'Issue',
            'numero': item.number,
            'titulo': item.title,
            'descripcion': item.body,
            'comentarios': " [SEP] ".join(comments),
            'total_comentarios': len(comments)
        })
        count += 1

    # ==========================================
    # 4. EXTRACCIÓN DE PULL REQUESTS (RF-01)
    # ==========================================
    print("\n--- Extrayendo Pull Requests ---")
    pulls = repo.get_pulls(state='all')
    count = 0

    for pr in pulls:
        if count >= LIMITE_REGISTROS: break

        print(f"📥 Procesando PR #{pr.number}...")
        comments = [c.body for c in pr.get_comments()]

        data.append({
            'tipo': 'Pull Request',
            'numero': pr.number,
            'titulo': pr.title,
            'descripcion': pr.body,
            'comentarios': " [SEP] ".join(comments),
            'total_comentarios': len(comments)
        })
        count += 1

    # ==========================================
    # 5. GUARDAR RESULTADOS
    # ==========================================
    df = pd.DataFrame(data)
    df.to_csv('datos_github_proyecto.csv', index=False, encoding='utf-8-sig')

    print(f"\n✨ ¡HECHO! Archivo 'datos_github_proyecto.csv' generado con {len(df)} filas.")

except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
files.download('datos_github_proyecto.csv')

In [ ]:
import nltk
import string
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
def preprocesado(documento):
  documento = documento.lower()
  tokens_nltk = nltk.word_tokenize(documento)
  tags = nltk.pos_tag(tokens_nltk)
  puntuacion = ['--', '_', ':', ')', "'", '"--', ',', ']', '!', '"', '?', '-', '--------', '[', ';', '.', '(', '----------', "'--", '&', "''", '``']
  tokens_nltk_limpio = [word for word, tag in tags if tag not in puntuacion]
  tokens_nltk_limpio = [t.strip('_') for t in tokens_nltk_limpio]
  if '[' in tokens_nltk_limpio: tokens_nltk_limpio.remove('[')
  if '[' in tokens_nltk_limpio: tokens_nltk_limpio.remove(']')
  stop_words = set(stopwords.words('spanish'))
  #stop_words.remove('no') para remover stop_words
  #stop_words.update('así') para añadir stop_words
  tokens_nltk_limpio = [w for w in tokens_nltk_limpio if w not in stop_words and w.isalnum()]
  del tokens_nltk, tags, puntuacion
  return tokens_nltk_limpio

In [ ]:
file_path = '/content/sample_data/datos_github_proyecto.csv'
#data = pd.read_csv(file_path, sep='\s+', header=None)
data = pd.read_csv(file_path)
data

,tipo,numero,titulo,descripcion,comentarios,total_comentarios
0,Issue,33154,Redundant execution of check_class_weight_bal...,### Describe the bug and give evidence about i...,@atheendre130505 \ncan i work on this ??\n,1
1,Issue,33152,⚠️ CI failed on Unit tests (last failure: Jan...,**CI failed on [Unit tests ](https://github.co...,Jan 29: `RuntimeError` in [Linux x86-64 pylat...,2
2,Issue,33146,Using `BisectingKMeans` with `init` raises a p...,"When using `BisectingKMeans`, `n_clusters` gre...","Thanks for the report, I think this is indeed ...",3
3,Issue,33143,RFC Metric `pos_label` handling for multiclass...,> [!WARNING]\n> This is a request for comment ...,NaN,0
4,Issue,33137,Analyze the practical relevance importance of ...,The following paper ran many hyper-parameter t...,NaN,0
...,...,...,...,...,...,...
95,Pull Request,33095,Added intutive explanation for linear regressi...,Added a plain -English sentence to the Ordinar...,NaN,0
96,Pull Request,33092,DOC: clarify that Lasso.path() defaults to Ela...,<!--\r\n🙌 Thanks for contributing a pull reque...,NaN,0
97,Pull Request,33091,DOC/MNT Little clean up around the splitting d...,Noticed while working on #33089\r\n\r\n#### Wh...,"I made the error message more specific, since ...",5
98,Pull Request,33090,CI remove scipy<1.16.2 requirement in wheel bu...,<!--\r\n🙌 Thanks for contributing a pull reque...,NaN,0


In [ ]:
data.count()

,0
tipo,100
numero,100
titulo,100
descripcion,100
comentarios,60
total_comentarios,100


In [ ]:
data.shape

(100, 6)